In [ ]:
'''
Introdução à Econometria - Uma abordagem moderna (Tradução da 6 edição norte-americana)
Autor: WOOLDRIDGE, J. M.
Editora: CENGAGE LEARNING

Cap. 8: Heterocedasticidade (Heteroskedasticity)
Exemplo 8.4: Heterocedasticidade nas equações de preços de imóveis
             (HETEROSKEDASTICITY IN HOUSING PRICE EQUATIONS)
             
Arquivo com os dados: hprice1.xls

Arquivo com dados em:
http://students.cengage.com.br/dashboard/private/livroView.jsf;jsessionid=95E9AD889A4A4B7ABBD2A5251F1E14BE?id=104577

Em caso de dúvidas ou problemas, solicitamos, por gentileza, entrar em contato pelo e-mail:
python.economia@gmail.com
'''

In [1]:
import pandas as pd
import statsmodels.formula.api as smf
import numpy as np
import statsmodels.stats.diagnostic

In [2]:
df = pd.read_excel('hprice1.xls',
                   header=None,
                   usecols=[0, 2, 3, 4],
                   names=['price', 'bdrms', 'lotsize', 'sqrft'])

In [3]:
df.head(5)

,price,bdrms,lotsize,sqrft
0,300.0,4,6126,2438
1,370.0,3,9903,2076
2,191.0,3,5200,1374
3,195.0,3,4600,1448
4,373.0,4,6095,2514


### Solução sugerida: Modelo 1

In [4]:
modelo = smf.ols('price ~ lotsize + sqrft + bdrms', data=df)
reg = modelo.fit()
reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.672
Model:                            OLS   Adj. R-squared:                  0.661
Method:                 Least Squares   F-statistic:                     57.46
Date:                Fri, 17 Jun 2022   Prob (F-statistic):           2.70e-20
Time:                        12:43:42   Log-Likelihood:                -482.88
No. Observations:                  88   AIC:                             973.8
Df Residuals:                      84   BIC:                             983.7
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    -21.7703     29.475     -0.739      0.462     -80.385      36.844
lotsize        0.0021      0.001      3.220      0.002       0.001       0.003
sqrft          0.1228      0.013      9.275      0.000       0.096       0.149
bdrms         13.8525      9.010      1.537      0.128      -4.065      31.770
==============================================================================
Omnibus:                       20.398   Durbin-Watson:                   2.110
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               32.278
Skew:                           0.961   Prob(JB):                     9.79e-08
Kurtosis:                       5.261   Cond. No.                     6.41e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 6.41e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [5]:
'''
Teste de Breusch-Pagan da heterocedasticidade (teste BP)
H0: O Erro u é Homocedástico
H1: Violação da homocedasticidade (presença de heterocedasticidade)
'''

teste_bp = statsmodels.stats.diagnostic.het_breuschpagan(reg.resid, exog_het=reg.model.exog)

print(f"estatística LM: {round(teste_bp[0], 2)}")
print(f"Valor p (LM): {round(teste_bp[1], 4)}")
print(f"estatística F: {round(teste_bp[2], 2)}")
print(f"Valor p (F): {round(teste_bp[3], 3)}")

estatística LM: 14.09
Valor p (LM): 0.0028
estatística F: 5.34
Valor p (F): 0.002


In [ ]:
'''
Conclusão: 
O baixo valor p (menor do que qualquer nível de significância padrão de 1%, 5% ou 10% ) é uma forte evidênncia contra H0.
Concluímos que o termo de erro apresenta heterocedasticidade.
Consequências imediatas da presença de heterocedasticidade
 - as colunas "std err", "t", "P>|t|" e [0.025 0.0975] da tabela com os resultados da regressão não são válidas
 - Não podemos utilizar as estatísticas t e F usuais para testar hipóteses
 - Os estimadores de MQO não são mais os melhores estimadores lineares não viesados
'''

### Solução sugerida: Modelo 2

In [6]:
modelo_2 = smf.ols('np.log(price) ~ np.log(lotsize) + np.log(sqrft) + bdrms', data=df)
reg_2 = modelo_2.fit()
reg_2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(price)   R-squared:                       0.643
Model:                            OLS   Adj. R-squared:                  0.630
Method:                 Least Squares   F-statistic:                     50.42
Date:                Fri, 17 Jun 2022   Prob (F-statistic):           9.74e-19
Time:                        12:43:51   Log-Likelihood:                 25.861
No. Observations:                  88   AIC:                            -43.72
Df Residuals:                      84   BIC:                            -33.81
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept          -1.2970      0.651     -1.992      0.050      -2.592      -0.002
np.log(lotsize)     0.1680      0.038      4.388      0.000       0.092       0.244
np.log(sqrft)       0.7002      0.093      7.540      0.000       0.516       0.885
bdrms               0.0370      0.028      1.342      0.183      -0.018       0.092
==============================================================================
Omnibus:                       12.060   Durbin-Watson:                   2.089
Prob(Omnibus):                  0.002   Jarque-Bera (JB):               34.889
Skew:                          -0.188   Prob(JB):                     2.65e-08
Kurtosis:                       6.062   Cond. No.                         410.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [ ]:
'''
Nota: No livro, por um erro, a estimativa do coeficiente de "bdrms" aparece como 0,37. O valor correto é 0,037.
'''

In [7]:
'''
Teste de Breusch-Pagan da heterocedasticidade (teste BP)
H0: O Erro u é Homocedástico
H1: Violação da homocedasticidade (presença de heterocedasticidade)
'''

teste_bp = statsmodels.stats.diagnostic.het_breuschpagan(reg_2.resid, exog_het=reg_2.model.exog)

print(f"estatística LM: {round(teste_bp[0], 2)}")
print(f"Valor p (LM): {round(teste_bp[1], 4)}")
print(f"estatística F: {round(teste_bp[2], 2)}")
print(f"Valor p (F): {round(teste_bp[3], 3)}")

estatística LM: 4.22
Valor p (LM): 0.2383
estatística F: 1.41
Valor p (F): 0.245


In [ ]:
'''
Conclusão: 
Não podemos rejeitar a  hipótese nula de homocedasticidade considerando-se qualquer nível de significância padrão de 1%, 5% 
ou 10% pois obtivemos um valor p MAIOR do que qualquer um destes valores.
'''